Some useful links from Dana:

https://scikit-image.org/docs/stable/auto_examples/edges/plot_skeleton.html

https://nalinc.github.io/blog/2018/skin-detection-python-opencv/

https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html

The paper and video we're basing everythign off of:

https://link.springer.com/article/10.1007/s11042-013-1501-1

https://www.youtube.com/watch?v=xML2S6bvMwI

In [11]:
import numpy as np
import math
import cv2
import random
import time


cap = cv2.VideoCapture(1)

# background = cv2.imread("image1.png")
# gun = cv2.imread("revolver2.png", cv2.IMREAD_UNCHANGED)
# bullet = cv2.imread("small_bullet.png", cv2.IMREAD_UNCHANGED)

# skin colour numbers
y_min = 64
y_max = 193
cr_min = 139 # 133
cr_max = 173
cb_min = 77
cb_max = 127

fire_ready = False
bullets = []
next_bullet = None
bullet_speed = 6
exit_angle = None
start_game = False
gameFinished = False
score = 0

win_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
win_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

centre1 = [random.randint(50,400), random.randint(175,250)]
centre2 = [random.randint(50,400), random.randint(300,450)]
centre3 = [random.randint(50,400), random.randint(500,680)]

start_time = 0;

# how long the game should last in seconds
game_length = 10;

# get the angle between ab and ac given a, b, c xy-points of a triangle
# uses the law of cosines aka the cosine rule / formula
def cos_rule(a, b, c):
    # euclidian distance between points
    ab = math.dist(a, b)
    bc = math.dist(b, c)
    ac = math.dist(a, c)

    # using law of cosines to compute angle between ab and bc
    angle_abac = math.degrees(math.acos((ab ** 2 + ac ** 2 - bc ** 2) / (2 * ab * ac)))
    return angle_abac

while 1:
    _ret, img = cap.read()
    #cv2.rectangle(img, (700, 700), (100, 100), (0, 255, 0), 0)
    crop_img = img#img[100:700, 100:700]
    drawing = np.zeros(crop_img.shape, np.uint8)
    #grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    kernel_size = (35, 35)
    
    img_ycrcb = cv2.cvtColor(crop_img, cv2.COLOR_BGR2YCrCb)
    #blurred = cv2.GaussianBlur(grey, kernel_size, 0)
    blurred = cv2.GaussianBlur(img_ycrcb, kernel_size, 0)

    scale_factor = 4
    if scale_factor != 1:
        blurred = cv2.resize(blurred, [blurred.shape[1]//scale_factor, blurred.shape[0]//scale_factor], interpolation=cv2.INTER_AREA)
    
    # skin color segmentation
    skin_ycrcb_mint = np.array([y_min, cr_min, cb_min], np.uint8)
    skin_ycrcb_maxt = np.array([y_max, cr_max, cb_max], np.uint8)
    thresholded = cv2.inRange(blurred, skin_ycrcb_mint, skin_ycrcb_maxt)
    #_ret, thresholded = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #contours, _hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # if no contours are detected, don't draw anything
    if len(contours) != 0:
        # this is the largest contour found, which helps eliminate noise and should be the hand
        count1 = max(contours, key=lambda x: cv2.contourArea(x))

        # drawing the biggest enclosing circle of the contour shape, used for gesture recognition
        big_c, big_r = cv2.minEnclosingCircle(count1)
        big_c = (int(big_c[0])*scale_factor, int(big_c[1])*scale_factor)

        #if ((big_r*scale_factor) > (3.5* center_dist*scale_factor)):
        # center point
        cv2.circle(crop_img, big_c, 1, (0, 0, 255), 2)
        # circle containing hand
        cv2.circle(crop_img, big_c, int(big_r)*scale_factor, (0, 0, 255), 2)
            
        x, y, w, h = cv2.boundingRect(count1)

        # finding the palm center
        center_pt = (0, 0)
        center_dist = 0
        for pt_x in range(x, x+w, 2):
            for pt_y in range(y, y+h, 2):
                pt = (pt_x, pt_y)
                dist = cv2.pointPolygonTest(count1, pt, measureDist=True)
                if dist > center_dist:
                    center_pt = (pt_x * scale_factor, pt_y * scale_factor)
                    center_dist = dist
                #cv2.circle(crop_img, pt, 1, (255, 0, 255), 1)
                #else:
                #    cv2.circle(crop_img, pt, 1, (0, 0, 0), 1)
        # biggest circle in palm
        cv2.circle(crop_img, center_pt, int(center_dist)*scale_factor, (255, 0, 0), 2)
        # circle 3.5 * the radius
        #cv2.circle(crop_img, center_pt, int(center_dist*3.5)*scale_factor, (255, 255, 0), 2)
        # biggest circle / 2
        cv2.circle(crop_img, center_pt, int(center_dist//2)*scale_factor, (0, 165, 255), 2)
        # center of biggest circle
        cv2.circle(crop_img, center_pt, 1, (255, 0, 0), 2)

        # not necessary at the moment
        # rotRect = cv2.minAreaRect(count1)
        # box = cv2.boxPoints(rotRect)
        # box = np.int64(box)
        # cv2.drawContours(crop_img, [box*scale_factor], 0, (255, 255, 0), 1)

        #cv2.rectangle(crop_img, (x*scale_factor, y*scale_factor), ((x + w)*scale_factor, (y + h)*scale_factor), (0, 0, 255), 1)

        hull = cv2.convexHull(count1)
        cv2.drawContours(drawing, [count1*scale_factor], 0, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull*scale_factor], 0, (0, 0, 255), 0)
        try:
            hull = cv2.convexHull(count1, returnPoints=False)
            defects = cv2.convexityDefects(count1, hull)
        except:
            # print(hull, count1)
            continue
            raise IOError("Hull issue")
        count_defects = 0
        cv2.drawContours(thresholded, contours*scale_factor, -1, (0, 255, 0), 3)
        

        # doing contour extractions once again within the 3.5x circle
        # grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        # blurred2 = cv2.GaussianBlur(grey, kernel_size, 0)
        # _ret, thresholded2 = cv2.threshold(blurred2, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        # contours2, _hierarchy = cv2.findContours(thresholded2.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#cv2.CHAIN_APPROX_NONE)

        # if len(contours2) != 0:
        #     count1_2 = max(contours2, key=lambda x: cv2.contourArea(x))


        # print(len(contours), defects.shape, '\n\n', contours)
        # break

        # go through all the defects in the hull
        if defects is not None:
            finger_tips = []
            finger_valleys = []
            angles = []
            thumb_defect = None
            biggest_angle = 0
            for defect in defects:
                # start index, end index, farthest pt index, depth
                s, e, f, d = defect[0]
                start = tuple(count1[s][0])
                end = tuple(count1[e][0])
                far = tuple(count1[f][0])

                # depth has 8 fractional bits, to get the value as a float divide by 256
                depth = d // 256

                angle = cos_rule(far, start, end)

                # cosidered a finger when the angle is < 90º and depth of defect is between the small and large radius
                if angle <= 100 and depth > center_dist and depth < big_r:
                    count_defects += 1
                    finger_tips.append([s, e, f])
                    angles.append(angle)
                    max(contours, key=lambda x: cv2.contourArea(x))

                    if angle > biggest_angle:
                        biggest_angle = angle
                        thumb_defect = [start, end, far]

                    cv2.circle(crop_img, (start[0]*scale_factor, start[1]*scale_factor), 3, [255, 0, 255], 3)
                    cv2.circle(crop_img, (end[0]*scale_factor, end[1]*scale_factor), 3, [255, 0, 255], 3)
                    cv2.circle(crop_img, (far[0]*scale_factor, far[1]*scale_factor), 3, [255, 0, 255], 3)
                    #print(f"defect {count_defects}: start: {start}, end: {end}, far: {far}, ra: {int(center_dist)}, l: {depth}, rb: {int(big_r)}")

                cv2.line(crop_img, (start[0]*scale_factor, start[1]*scale_factor), (end[0]*scale_factor, end[1]*scale_factor), [0, 255, 0], 2)

            # for a in range(len(angles)):
            biggest_area = 0
            thumb = None

            for finger_tip in finger_tips:
                s, e, f = finger_tip
                area = cv2.contourArea(count1[s:e+1])
                if area > biggest_area:
                    biggest_area = area
                    thumb = finger_tip
                #biggest_area = max(finger_tips, key=lambda x: cv2.contourArea(count1[x[0]:x[1]+1]))
            
            if thumb is not None and start_game:
                s, e, f = thumb
                count1_reshaped = count1.reshape(-1, 2)

                cv2.fillPoly(crop_img, [count1_reshaped[s:e+1]*scale_factor], (255, 0, 0, 0.2))
                
                start = count1[s][0]
                end = count1[e][0]
                far = count1[f][0]
                angle = cos_rule(far, start, end)

                tip_1 = math.dist(start, far)
                tip_2 = math.dist(end, far)

                if (tip_1 > tip_2):
                    pointer_tip = start
                else:
                    pointer_tip = end

                cv2.line(crop_img, pointer_tip*scale_factor, far*scale_factor, (0, 0, 255), 6)

                # if angle < 80:
                #     fire_ready = False
                # else:
                cv2.putText(img, f"angle: {angle:3.0f} - aim", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)
                fire_ready = True
                next_bullet = pointer_tip

                x_dist = pointer_tip[0] - far[0]
                y_dist = pointer_tip[1] - far[1]
                exit_angle = math.atan2(y_dist, x_dist)

        if fire_ready and count_defects == 0:
            cv2.putText(img, f"Fire!", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)
            fire_ready = False

            if next_bullet is not None:
                bullets.append([next_bullet, exit_angle])

                next_x = next_bullet[0] + math.cos(exit_angle) * bullet_speed
                next_y = next_bullet[1] + math.sin(exit_angle) * bullet_speed
                next_coords = (next_x, next_y)
                #print(f"bullet_coords: {next_bullet}, angle: {math.degrees(exit_angle)}, next_coords: {next_coords}")

        if count_defects < 5:
            cv2.putText(img, f"{count_defects+1} finger(s)    Score: {score}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)
            
        if not start_game and count_defects == 4:
            start_game = True
            start_time = time.time()

    new_bullets = []

    target1 = cv2.circle(crop_img, centre1 , 30, [255, 0, 0], -1)
    target2 = cv2.circle(crop_img, centre2 , 30, [0, 255, 0], -1)
    target3 = cv2.circle(crop_img, centre3 , 30, [0, 0, 255], -1)

    for b in range(len(bullets)):

        bullet_coord, bullet_angle = bullets[b]
        print(bullet_coord)

        # target 1 collision detection
        if (math.dist(bullet_coord*scale_factor, centre1) < 30):
                print("HIT")
                target1 = cv2.circle(crop_img, centre1, 40, (0, 0, 0), -1)
                centre1 = [random.randint(50,400), random.randint(175,250)]

        # target 2 collision detection
        elif (bullet_coord[0]== centre2[0]) and (bullet_coord[1] == centre2[1]):
                print("HIT")
                target2 = cv2.circle(crop_img, centre2, 40, (0, 0, 0), -1)
                centre2 = [random.randint(50,400), random.randint(300,450)]

        # target 3 collision detection
        elif (bullet_coord[0]== centre3[0]) and (bullet_coord[1] == centre3[1]):
                print("HIT")
                target3 = cv2.circle(crop_img, centre3, 40, (0, 0, 0), -1)
                centre3 = [random.randint(50,400), random.randint(500,680)]

        # get rid of any bullets not within the screen
        if bullet_coord[0] > 0 and bullet_coord[0] < crop_img.shape[1] and bullet_coord[1] > 0 and bullet_coord[1] < crop_img.shape[0]:
            cv2.circle(crop_img, bullet_coord*scale_factor, 5, (255, 255, 120), 3)
            cv2.circle(drawing, bullet_coord*scale_factor, 5, (255, 255, 120), 3)

            # update the bullet location for the next frame
            bullet_coord[0] += math.cos(bullet_angle) * bullet_speed
            bullet_coord[1] += math.sin(bullet_angle) * bullet_speed
            new_bullets.append([bullet_coord, bullet_angle])

    bullets = new_bullets

    
    if start_game:
        elapsed_time = time.time() - start_time
        time_left = game_length - elapsed_time
        if gameFinished or time_left <= 0:
            gameFinished = True
            cv2.putText(img, "Finished!", (50, win_h - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)
        elif time_left <= 5:
            cv2.putText(img, f"Time: {time_left:.2f}", (50, win_h - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 100), 4)
        else:
            cv2.putText(img, f"Time: {time_left:.2f}", (50, win_h - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 4)

        cv2.putText(img, "game started", (50, win_h - 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 100, 0), 4)

    else:
        cv2.putText(img, "waiting", (50, win_h - 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 180), 4)

    #cv2.imshow('main window', img)

    all_img = np.vstack((crop_img,drawing))

    cv2.imshow('Contours', all_img)
    #print('\n')
    #input()

    k = cv2.waitKey(1)
    if k == 27:
        break






cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()



[156 101]
[150 101]
[144 101]
[138 101]
[132 101]
[151 115]
[126 101]
[145 115]
[120 101]
[139 115]
[114 101]
[133 115]
[108 101]
[127 115]
[102 101]
[121 115]
[151  99]
[ 96 101]
[115 115]
[145  99]
[ 90 101]
[109 115]
[139  99]
[ 84 101]
[103 115]
[133  99]
[ 78 101]
[ 97 115]
[127  99]
[141  90]
[ 72 101]
[ 91 115]
[121  99]
[135  90]
[ 66 101]
[ 85 115]
[115  99]
[129  90]
[ 60 101]
[ 79 115]
[109  99]
[123  90]
[ 54 101]
[ 73 115]
[103  99]
[117  90]
[ 48 101]
[ 67 115]
[97 99]
[111  90]
[137  73]
[ 42 101]
[ 61 115]
[91 99]
[105  90]
[131  73]
[ 36 101]
[ 55 115]
[85 99]
[99 90]
[125  73]
[ 30 101]
[ 49 115]
[79 99]
[93 90]
[119  73]
[ 24 101]
[ 43 115]
[73 99]
[87 90]
[113  73]
[140  63]
[ 18 101]
[ 37 115]
[67 99]
[81 90]
[107  73]
[134  63]
[ 12 101]
[ 31 115]
[61 99]
[75 90]
[101  73]
[128  63]
[  6 101]
[ 25 115]
[55 99]
[69 90]
[95 73]
[122  63]
[  0 101]
[ 19 115]
[49 99]
[63 90]
[89 73]
[116  63]
[ 13 115]
[43 99]
[57 90]
[83 73]
[110  63]
[172  74]
[  7 115]
[37 99]
[51 